# Performance: AUROC Barplots
## Lawrence He and Felipe Giuste (2022-09-09)

In [1]:
import sys, os
import numpy as np
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

## Classifiers ##
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from joblib import load

## Metrics ##
from sklearn.metrics import roc_auc_score, matthews_corrcoef, f1_score, accuracy_score, recall_score, precision_score
from scipy.stats import wilcoxon 

### Seed ###
random_state = 1234
np.random.seed(random_state)

## User Variables

In [7]:
## Folder paths ##
data_folder = '../Data/'
results_folder='../Results/'
models_folder='../Models'
figures_folder ='../Figures/Performance/'

## Outcome variable ##
outcome_column='ckd_status'
## Feature Ranks ##

## Risk Threshold Increment ## 
threshold_increment=1e-3
thresholds = np.arange(0, 1+threshold_increment, threshold_increment)
n_thresholds = len(thresholds)

## Test Performance ## 
performance_dict = {}

In [3]:
### Deep Learning Model Classifiers ###
deep_model_names = [
    'relu',
    'softmax',
    'sigmoid',
    'gumbel_softmax',
]

## Conventional Classifier dictionary ##
conventional_classifier_dict={'LogisticRegression':LogisticRegression, 
                 'RandomForestClassifier':RandomForestClassifier, 
                 'AdaBoostClassifier':AdaBoostClassifier, 
                 'GaussianNB':GaussianNB, 
                 'KNeighborsClassifier':KNeighborsClassifier, 
                 'SVC_radial':SVC, 
                 'XGBoostClassifier':XGBClassifier
                }
conventional_model_names = list(conventional_classifier_dict.keys())

## Cleanup model names ##
model_name_map = {'LogisticRegression':'Logistic Regression',
                  'RandomForestClassifier':'Random Forest',
                  'AdaBoostClassifier':'AdaBoost Classifier',
                  'GaussianNB':'Gaussian NB',
                  'KNeighborsClassifier':'KNN',
                  'SVC_radial':'SVC',
                  'XGBoostClassifier':'XGBoost',
                  'relu':'NN: ReLu',
                  'softmax':'NN: Softmax',
                  'sigmoid':'NN: Sigmoid',
                  'gumbel_softmax':'NN: Gumbel Softmax',
}

## Load Data

In [4]:
### Load ###
train_data = pd.read_csv(data_folder+'train_data.csv')
test_data = pd.read_csv(data_folder+'test_data.csv')

### Feature list ###
feature_list = list()
for i in test_data.columns:
    if i[:2] == 'F_':
        feature_list.append(i)
        
### Setup Datasets ###
X_train = train_data[feature_list]#.rename(columns=feature_name_dict)
y_train = train_data[outcome_column]
X_test = test_data[feature_list]#.rename(columns=feature_name_dict)
y_test = test_data[outcome_column]

## Update Feature List ##
feature_list = list(X_test.columns)
print('Total Features: %s'%len(feature_list) )

## Get Top-10 Features ##
# with open(feature_ranks_path, 'rb') as fh:
#     f_rankings_dict= pickle.load(fh)
# features_top10 = np.array(f_rankings_dict['Feature Ranks'][:10])
# features_top10

## List of feature subsets ##
feature_subsets = np.array(['All Features'])

/var/folders/65/qxp2fcwj00bff0rdg3hmhyx80000gn/T/ipykernel_66602/2873637929.py:2: DtypeWarning: Columns (30,41,42,66,67,68,69,83,87,103,104,105,121,122,123) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(data_folder+'train_data.csv')


Total Features: 128


/var/folders/65/qxp2fcwj00bff0rdg3hmhyx80000gn/T/ipykernel_66602/2873637929.py:3: DtypeWarning: Columns (30,41,42,66,67,68,69,83,87,103,104,105,121,122,123) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(data_folder+'test_data.csv')


## Generate model performance on Test dataset

In [1]:
## Skip if Results Exist ##
if(not os.path.isfile(f'{models_folder}/Test_Performance.pkl')):

    ## Iterate across feature subsets ##
    for feature_subset in feature_subsets:
        
        ## Iterate across Classifiers ##
        for model_name in model_name_map.keys():
            print(f'{feature_subset} : {model_name}          ')
            
            ## Dictionary keys ##
            if(model_name not in performance_dict):
                performance_dict[model_name] = {}
            if(feature_subset not in performance_dict[model_name]):
                performance_dict[model_name][feature_subset] = {}
        
            ## Ablated Dataset ##
            # if(feature_subset in features_top10):
            #     ## Ablate Feature from Top-10 ##
            #     ablated_feature_list = features_top10[features_top10 != feature_subset]
            #     assert( len(ablated_feature_list) == (len(features_top10)-1) )
            #     ## Feature Ablation ##
            #     X_train_subset = X_train[ablated_feature_list]
            #     X_test_subset = X_test[ablated_feature_list]
            #el
            if(feature_subset == 'All Features'):
                ## All ##
                X_train_subset = X_train
                X_test_subset= X_test
            # elif(feature_subset == 'Top10 Features'):
            #     ## Top-10 ##
            #     X_train_subset = X_train[features_top10]
            #     X_test_subset = X_test[features_top10]
            else:
                raise Exception('Unknown Feature Subset: %s'% feature_subset)
            ## Check that features do not contain outcome variable ##
            assert( outcome_column not in list(X_train_subset.columns) )
            assert( outcome_column not in list(X_test_subset.columns) )
            ## Update Feature List ##
            feature_list = list(X_test_subset.columns)

            ## Conventional Model ##
            if(model_name in conventional_model_names):
                ## Model Load ##
                model_path = f'{models_folder}/Conventional/{model_name}.'
                model = load(model_path) 

                ## Find best threshold for prediction binarization (on Training dataset) ##
                print('\tPredict ', end='\r')
                y_pred = model.predict_proba(X_train_subset)[:,1]                
                print(f'\tThresholding...', end='\r')
                mcc_list = [ matthews_corrcoef(y_true=y_train, y_pred=y_pred>thresh) for thresh in thresholds ]
                best_thresh = thresholds[np.where( mcc_list == np.max(mcc_list) )[0]][0]
                print(f'Best Threshold: {best_thresh}       ')
                
                ## Test ##
                print('\tTesting ', end='\r')
                y_pred = model.predict_proba(X_test_subset)[:,1]
                y_pred_bin = y_pred > best_thresh
                performance_dict[model_name][feature_subset]['y_pred'] = y_pred

            ## Deep Model ##
            # # # elif(model_name in deep_model_names):
            # #     ## Model ID ##
            # #     # if(feature_subset in features_top10):
            # #     #     model_id = f'FABLATION-{feature_subset}'
            # #     #el
            # #     if(feature_subset == 'All Features'):
            # #         model_id = 'ALLFEATURES'
            # #     # elif(feature_subset == 'Top10 Features'):
            # #     #     model_id = 'TOP10'
            # #     else:
            # #         raise('Unknown Feature Subset: %s'% feature_subset)

            # #     ## File with Deep Learning models HP Tuning Information ##
            # #     # hptune_df = pd.read_csv(hptune_csv)
            # #     # ## Load Model Hyperparameters from HPTune dataframe ##
            # #     # hyperparameters_of_interest = hptune_df[(hptune_df['penultimate_activation_type']==model_name) & 
            # #     #                                         (hptune_df['optimized_model']==True) & 
            # #     #                                         (hptune_df['outcome_variable']==outcome_column)]
            # #     # ## Setup Hyperparameter Values ##
            # #     # dropout1_p = hyperparameters_of_interest['dropout1_p'].values[0]
            # #     # dropout2_p = hyperparameters_of_interest['dropout2_p'].values[0]
            # #     # dropout3_p = hyperparameters_of_interest['dropout3_p'].values[0]
            # #     # clustering_neurons = hyperparameters_of_interest['clustering_neurons'].values[0]
            # #     # learning_rate = hyperparameters_of_interest['learning_rate'].values[0]
            # #     # outcome_variable = hyperparameters_of_interest['outcome_variable'].values[0]
            # #     # ## Model File ##
            # #     # model_file=f'{model_id}_ACTIVATION={model_name}_CN={clustering_neurons}_D1P={dropout1_p}_D2P={dropout2_p}_D3P={dropout3_p}_LR={learning_rate}_OUTCOME={outcome_variable}'
            # #     # ## Feature Layer Size ##
            # #     # input_neurons=len(feature_list)
            # #     # ## Model Load ##
            # #     # model = DeepLearningArchitecture(dropout1_p=dropout1_p, dropout2_p=dropout2_p, dropout3_p=dropout3_p,
            # #     #                                            clustering_neurons=clustering_neurons,
            # #     #                                            penultimate_activation_type=model_name, input_neurons=input_neurons)
            # #     # model.load_state_dict(torch.load(f'{models_folder}/Deep/Testing-100Trained/{model_file}_checkpoint.pth', 
            # #     #                                  map_location=device )
            # #     #                      )
            # #     # model.to(device)
            # #     # model.eval()
                
            # #     ## Predict ##
            # #     print('\tPredict ', end='\r')
            # #     y_pred = model( torch.from_numpy(X_train_subset.values).float().to(device) ).detach().cpu().numpy()
            # #     ## Find best threshold for prediction binarization (on Training dataset) ##
            # #     print(f'\tThresholding...', end='\r')
            # #     mcc_list = [ matthews_corrcoef(y_true=y_train, y_pred=y_pred>thresh) for thresh in thresholds ]
            # #     best_thresh = thresholds[np.where( mcc_list == np.max(mcc_list) )[0]][0]
            # #     print(f'Best Threshold: {best_thresh}       ')
                

            # #     ## Test ##
            # #     print('\tTesting ', end='\r')
            # #     y_pred = model( torch.from_numpy(X_test_subset.values).float().to(device) ).detach().cpu().numpy()
            # #     y_pred_bin = y_pred > best_thresh
            # #     performance_dict[model_name][feature_subset]['y_pred'] = y_pred
                
            ## Performance ##
            performance_dict[model_name][feature_subset]['threshold'] = best_thresh
            performance_dict[model_name][feature_subset]['AUROC'] = roc_auc_score(y_true=y_test, y_score=y_pred)
            performance_dict[model_name][feature_subset]['MCC'] = matthews_corrcoef(y_true=y_test, y_pred=y_pred_bin)
            performance_dict[model_name][feature_subset]['F1'] = f1_score(y_true=y_test, y_pred=y_pred_bin)
            performance_dict[model_name][feature_subset]['accuracy'] = accuracy_score(y_true=y_test, y_pred=y_pred_bin)
            performance_dict[model_name][feature_subset]['recall'] = recall_score(y_true=y_test, y_pred=y_pred_bin)
            performance_dict[model_name][feature_subset]['precision'] = precision_score(y_true=y_test, y_pred=y_pred_bin)
            ## Delete model ##
            del(model)
            
    ## Save Performance Dictionary ##
    with open( f'{models_folder}/Test_Performance.pkl', 'wb' ) as fh:
         pickle.dump( performance_dict, file=fh )
else:
    print(f'Exists: {models_folder}/Test_Performance.pkl')
    
print('Done')

NameError: name 'models_folder' is not defined